In [1]:
from fishsense_api_sdk.client import Client
from tqdm.asyncio import tqdm_asyncio
from typing import List
from fishsense_api_sdk.models.species_label import SpeciesLabel
from synology_api.filestation import FileStation
from fishsense_data_processing_workflow_worker.config import settings
from pathlib import Path
from tqdm.notebook import tqdm
from skimage.exposure import adjust_gamma, equalize_adapthist
from skimage.util import img_as_float, img_as_ubyte
import rawpy
import cv2
import numpy as np
import pymupdf
import math

In [2]:
URL = "http://localhost:8000"

NAS_HOST = "e4e-nas.ucsd.edu"
NAS_PORT = 6021

In [3]:
DATA_FOLDER = (Path("../data") / "REEF" / "data").absolute()
OUTPUT_FOLDER = (Path("../output") / "preprocess_slate_images_jpeg").absolute()

DATA_FOLDER.mkdir(parents=True, exist_ok=True)
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

DATA_FOLDER.exists(), OUTPUT_FOLDER.exists()

(True, True)

In [4]:
filestation = FileStation(NAS_HOST, NAS_PORT, settings.e4e_nas.username, settings.e4e_nas.password, secure=True, cert_verify=False)

User logged in, new session started!


In [5]:
async with Client(URL) as fs:
    dives = await fs.dives.get_canonical()

len(dives), dives[0]

(272,
 Dive(id=1, name='080123_FSL-01 Photos', path='2023-09-07 REEF Data Dump/080123_FSL-01 Photos', dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27, tzinfo=TzInfo(0)), priority=<Priority.LOW: 'LOW'>, flip_dive_slate=True, camera_id=1, dive_slate_id=1))

In [6]:
high_priority_dives = [dive for dive in dives if dive.priority == 'HIGH']

len(high_priority_dives), high_priority_dives[0]

(7,
 Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=7))

In [7]:
high_priority_dives_by_id = {dive.id: dive for dive in high_priority_dives}

high_priority_dives_by_id

{279: Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=7),
 341: Dive(id=341, name='031424_Alligator0_FSL01', path='2024.06.20.REEF/03_2024_Alligator/031424_Alligator_FSL01/031424_Alligator0_FSL01', dive_datetime=datetime.datetime(2024, 3, 13, 23, 59, 45, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=1, dive_slate_id=8),
 347: Dive(id=347, name='031424_Alligator0_FSL02', path='2025-02-10 REEF Data Dump SMILE 6/031424_Alligator_FSL02/031424_Alligator0_FSL02', dive_datetime=datetime.datetime(2024, 3, 14, 12, 15, 46, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=2, dive_slate_id=11),
 349: Dive(id=349, name='03142

In [8]:
async with Client(URL) as client:
    camera_intrinsics_list = await tqdm_asyncio.gather(*[client.cameras.get_intrinsics(dive.camera_id) for dive in high_priority_dives])

len(camera_intrinsics_list), camera_intrinsics_list[0]

100%|██████████| 7/7 [00:01<00:00,  5.51it/s]


(7,
 <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a2988b00>)

In [9]:
camera_intrinsics_by_dive_id = {dive.id: intrinsics for dive, intrinsics in zip(high_priority_dives, camera_intrinsics_list)}

camera_intrinsics_by_dive_id

{279: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a2988b00>,
 341: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a328aa50>,
 347: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a32670b0>,
 349: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a2962e90>,
 465: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a3208e10>,
 466: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a32a5ba0>,
 471: <fishsense_api_sdk.models.camera_intrinsics.CameraIntrinsics at 0x7e70a32638a0>}

In [10]:
species_labels: List[SpeciesLabel] = await tqdm_asyncio.gather(*[fs.labels.get_species_labels(dive.id) for dive in high_priority_dives])
species_labels = [label for sublist in species_labels for label in sublist]

len(species_labels), species_labels[0]

100%|██████████| 7/7 [00:02<00:00,  2.85it/s]


(1178,
 SpeciesLabel(id=1024, label_studio_task_id=222252, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/7252dfb0db0b855616e45607f58c1c93', updated_at=datetime.datetime(2025, 10, 20, 20, 42, 14, 832143, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=True, laser_x=1970.0, laser_y=1340.0, laser_label='Red Laser', content_of_image='Slate, Laser not on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 117942, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 49.08221189998697, 'y': 44.44128413294742, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction-changed', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'm4K68yydiZ', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser not on

In [11]:
slate_labels = [label for label in species_labels if label.content_of_image == 'Slate, Laser on slate']

len(slate_labels), slate_labels[0]

(95,
 SpeciesLabel(id=1044, label_studio_task_id=222272, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/2863b55ffedf2bbb1e94f8042fd8ad60', updated_at=datetime.datetime(2025, 10, 21, 1, 8, 15, 395055, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=True, laser_x=1851.0, laser_y=902.0, laser_label='Red Laser', content_of_image='Slate, Laser on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 117986, 'result': [{'id': 'result1', 'type': 'keypointlabels', 'value': {'x': 46.10335466840985, 'y': 29.916128498866595, 'width': 0.2, 'keypointlabels': ['Red Laser']}, 'origin': 'prediction', 'to_name': 'image', 'from_name': 'laser', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}, {'id': 'xON5aKx1k3', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser on slate']]}, 'origin'

In [12]:
async with Client(URL) as fs:
    slate_images = await tqdm_asyncio.gather(*[fs.images.get(image_id=label.image_id) for label in slate_labels])

len(slate_images), slate_images[0]

100%|██████████| 95/95 [00:02<00:00, 32.93it/s]


(95,
 Image(id=78686, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130190.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 52, tzinfo=TzInfo(0)), checksum='2863b55ffedf2bbb1e94f8042fd8ad60', is_canonical=True, dive_id=279, camera_id=5))

In [13]:
def process_raw(image_path: Path):
    with image_path.open("rb") as f:
        with rawpy.imread(f) as raw:
            img = img_as_float(
                raw.postprocess(
                    gamma=(1, 1),
                    no_auto_bright=True,
                    use_camera_wb=True,
                    output_bps=16,
                    user_flip=0,
                )
            )

            hsv = cv2.cvtColor(img_as_ubyte(img), cv2.COLOR_BGR2HSV)
            _, _, val = cv2.split(hsv)

            mid = 20
            mean = np.mean(val)
            meanLog = math.log(mean)
            midLog = math.log(mid * 255)
            gamma = midLog / meanLog
            gamma = 1 / gamma

            img = adjust_gamma(img, gamma=gamma)

            img = equalize_adapthist(img)

            return img_as_ubyte(img[:, :, ::-1])

In [14]:
def rectify(img: np.ndarray, camera_matrix: np.ndarray, distortion_coefficients: np.ndarray) -> np.ndarray:
    return cv2.undistort(
        img,
        camera_matrix,
        distortion_coefficients,
    )

In [15]:
async with Client(URL) as fs:
    dive_slates = await client.dive_slates.get()

len(dive_slates), dive_slates[0]

(11,
 DiveSlate(id=5, name='Tic-Tac-Toe 4', dpi=300, path='Dive Slate#4.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 190057, tzinfo=TzInfo(0)), reference_points=[(591.9493127134886, 419.73291956851295), (1180.375218367864, 415.508647477124), (2008.7060450606716, 408.7832988960453), (2598.7223535242415, 401.20588324117523), (594.2177470825221, 1320.1595244257721), (1183.4343179108548, 1318.369903930235), (2016.1826073962275, 1316.5857288918276), (2604.907191981395, 1314.7051789793095)]))

In [16]:
dive_slates_by_id = {dive_slate.id: dive_slate for dive_slate in dive_slates}

dive_slates_by_id

{5: DiveSlate(id=5, name='Tic-Tac-Toe 4', dpi=300, path='Dive Slate#4.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 190057, tzinfo=TzInfo(0)), reference_points=[(591.9493127134886, 419.73291956851295), (1180.375218367864, 415.508647477124), (2008.7060450606716, 408.7832988960453), (2598.7223535242415, 401.20588324117523), (594.2177470825221, 1320.1595244257721), (1183.4343179108548, 1318.369903930235), (2016.1826073962275, 1316.5857288918276), (2604.907191981395, 1314.7051789793095)]),
 6: DiveSlate(id=6, name='Tic-Tac-Toe 5', dpi=300, path='Dive Slate #5.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 190162, tzinfo=TzInfo(0)), reference_points=[(403.8205816493262, 631.5968154288058), (993.2497902769651, 632.7766211186832), (1740.9198685182203, 635.3794945908605), (2328.4124741771443, 635.521328189681), (425.52889197228865, 1488.5791173820157), (1014.7334760788126, 1491.8682245487987), (1758.603030468457, 1496.092149318488), (2346.7834576540045, 1498.101145176125

In [17]:
dive_slates_by_dive_id = {dive.id: dive_slates_by_id[dive.dive_slate_id] for dive in high_priority_dives}

dive_slates_by_dive_id

{279: DiveSlate(id=7, name='Tic-Tac-Toe 6', dpi=300, path='Dive Slate#6.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 190259, tzinfo=TzInfo(0)), reference_points=[(495.4112011445373, 392.73624451189136), (1085.311077010467, 389.554881713577), (1759.819487534749, 385.38891892656096), (2348.1289135916677, 379.9113269488712), (548.2033935446091, 1545.108970466092), (1137.1556910423612, 1539.6503159681731), (1772.0744635842048, 1535.2580570522443), (2360.6314833201213, 1529.5025686051904)]),
 341: DiveSlate(id=8, name='V-Slate 1', dpi=300, path='SMILE vslate 1.pdf', created_at=datetime.datetime(2025, 9, 6, 3, 53, 10, 190350, tzinfo=TzInfo(0)), reference_points=[(1273.415553462368, 261.32005259514614), (2524.08142156518, 1592.7326951022442), (3645.651741698752, 556.3806989055934), (4045.648281376697, 965.3555721819871), (2504.9828552161666, 2396.673231186069), (863.4172026886467, 654.9408839549113)]),
 347: DiveSlate(id=11, name='V-Slate 4', dpi=300, path='SMILE vslate 4.pdf', c

In [18]:
for image in tqdm(slate_images):
    dive = high_priority_dives_by_id[image.dive_id]
    dive_slate = dive_slates_by_dive_id[dive.id]
    camera_intrinsics = camera_intrinsics_by_dive_id[dive.id]

    image_path = DATA_FOLDER / image.path
    pdf_path = DATA_FOLDER / dive_slate.path
    target_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

    source_nas_path = f"/fishsense/Fishsense Lite Calibration Parameters/{dive_slate.path}"
    print(f"Downloading {source_nas_path} to {pdf_path}")
    filestation.get_file(source_nas_path, "download", dest_path=str(pdf_path.parent))

    with pymupdf.open(pdf_path) as pdf_document:
        page: pymupdf.Page = pdf_document.load_page(0)
        pixmap: pymupdf.Pixmap = page.get_pixmap(dpi=dive_slate.dpi)
        bytes = np.frombuffer(pixmap.samples, dtype=np.uint8)

        pdf_image = bytes.reshape(pixmap.height, pixmap.width, pixmap.n)
        pdf_image = cv2.cvtColor(pdf_image, cv2.COLOR_RGB2GRAY)

        _, pdf_image = cv2.threshold(pdf_image, 125, 255, cv2.THRESH_BINARY)

        pdf_image = cv2.cvtColor(pdf_image, cv2.COLOR_GRAY2BGR)

    source_nas_path = f"/fishsense_data/REEF/data/{image.path}"
    filestation.get_file(source_nas_path, "download", dest_path=str(image_path.parent))

    img = process_raw(image_path)
    img = rectify(img, camera_intrinsics.camera_matrix, camera_intrinsics.distortion_coefficients)

    img_height, img_width = img.shape[:2]
    pdf_height, pdf_width = pdf_image.shape[:2]

    scale_y = float(img_height) / float(pdf_height)

    pdf_height = int(pdf_height * scale_y)
    pdf_width = int(pdf_width * scale_y)
    pdf_image = cv2.resize(pdf_image, (pdf_width, pdf_height))

    new_img = np.zeros((img_height, img_width + pdf_width, 3), dtype=np.uint8)
    new_img[:, :pdf_width, :] = pdf_image
    new_img[:, pdf_width:, :] = img

    for idx, point in enumerate(dive_slate.reference_points):
        x, y = point
         
        x*= scale_y
        y*= scale_y

        x = int(x)
        y = int(y)

        cv2.circle(new_img, (x, y), radius=25, color=(0, 0, 255), thickness=-1)

        text = f"{idx + 1}"
        org = (x + 20, y - 10)
        fontFace = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 5
        color = (0, 0, 255) # Red color (BGR)
        thickness = 10
        lineType = cv2.LINE_AA

        # Put the text on the image
        cv2.putText(new_img, text, org, fontFace, fontScale, color, thickness, lineType)

    cv2.imwrite(target_path.as_posix(), new_img)

    target_nas_path = f"/fishsense_process_work/preprocess_slate_images_jpeg"
    filestation.upload_file(target_nas_path, str(target_path), overwrite=True)

  0%|          | 0/95 [00:00<?, ?it/s]

Upload Progress: 100%|██████████| 6.46M/6.46M [00:09<00:00, 719kB/s]


Upload Progress: 100%|██████████| 6.49M/6.49M [00:09<00:00, 727kB/s]


Upload Progress: 100%|██████████| 6.51M/6.51M [00:09<00:00, 737kB/s]


Upload Progress: 100%|██████████| 6.97M/6.97M [00:08<00:00, 856kB/s] 


...

Upload Progress: 100%|██████████| 7.60M/7.60M [00:05<00:00, 1.56MB/s]


Upload Progress: 100%|██████████| 7.48M/7.48M [00:05<00:00, 1.44MB/s]


Upload Progress: 100%|██████████| 7.52M/7.52M [00:05<00:00, 1.49MB/s]


Upload Progress: 100%|██████████| 7.69M/7.69M [00:05<00:00, 1.48MB/s]


Upload Progress: 100%|██████████| 7.48M/7.48M [00:05<00:00, 1.46MB/s]
